### Joins
---------
The material in this notebook was extracted from
* Spark The Definitive Guide Big Data Processing Made Simple (2018)
---------

A join brings together two sets of data, the left and the right, by comparing the value of one or more *keys*: if they are equal, Spark will combine the left and right datasets. The opposite is true for keys that do not match; Spark discards the rows that do not have matching keys. 

##### Join Types

Whereas the join expression determines whether two rows should join, the join type determines what should be in the result set. There are various join types available in Spark:

* Inner joins (keep rows with keys that exist in the left and right datasets)
* Outer joins (keep rows with keys in either the left or right datasets)
* Left semi joins (keep the rows of the left dataset where the key appears in the right dataset)
* Left anti joins (keep the rows of the left dataset where the key doesn't appear in the right dataset)
* Natural joins (perform a join by implicitly matching the columns between the two datasets with the same names)
* Cross joins (match every row in the left dataset with every row in the right dataset)

In [1]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matel Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100]),
    (3, "Luis", 4, [250, 100, 1000])
]).toDF("id", "name", "graduate_program", "spark_status")
program = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkley"),
    (1, "Masters", "EECS", "UC Berkley"),
    (2, "Ph.D.", "EECS", "UC Berkley")
]).toDF("id", "degree", "department", "school")
status = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"),
    (1000, "CEO")
]).toDF("id", "status")

##### Inner Joins

Inner joins evaluate keys in both DataFrames and include only the rows that evaluate to true.

In [2]:
joinCondition = person['graduate_program'] == program['id']

In [3]:
person.join(program, joinCondition).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



Or more explicitly 

In [4]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'inner').show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



##### Outer Joins

Outer joins evaluate the keys in both of the DataFrames and includes the rows that evaluate true or false. If there is no eequivalent row in either left or right DataFrame, Spark will insert null

In [5]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'left_outer').show()

+---+----------------+----------------+----------------+----+-------+--------------------+----------+
| id|            name|graduate_program|    spark_status|  id| degree|          department|    school|
+---+----------------+----------------+----------------+----+-------+--------------------+----------+
|  0|   Bill Chambers|               0|           [100]|   0|Masters|School of Informa...|UC Berkley|
|  1|   Matel Zaharia|               1| [500, 250, 100]|   1|Masters|                EECS|UC Berkley|
|  2|Michael Armbrust|               1|      [250, 100]|   1|Masters|                EECS|UC Berkley|
|  3|            Luis|               4|[250, 100, 1000]|null|   null|                null|      null|
+---+----------------+----------------+----------------+----+-------+--------------------+----------+



In [6]:
program.show()
person.show()
person.join(program, 
            person['graduate_program'] == program['id'], 
            'left_semi').show()

+---+-------+--------------------+----------+
| id| degree|          department|    school|
+---+-------+--------------------+----------+
|  0|Masters|School of Informa...|UC Berkley|
|  1|Masters|                EECS|UC Berkley|
|  2|  Ph.D.|                EECS|UC Berkley|
+---+-------+--------------------+----------+

+---+----------------+----------------+----------------+
| id|            name|graduate_program|    spark_status|
+---+----------------+----------------+----------------+
|  0|   Bill Chambers|               0|           [100]|
|  1|   Matel Zaharia|               1| [500, 250, 100]|
|  2|Michael Armbrust|               1|      [250, 100]|
|  3|            Luis|               4|[250, 100, 1000]|
+---+----------------+----------------+----------------+

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [10

In [7]:
person.join(program, 
            person['graduate_program'] == program['id'], 
            'right_outer').show()

+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkley|
|   1|   Matel Zaharia|               1|[500, 250, 100]|  1|Masters|                EECS|UC Berkley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters|                EECS|UC Berkley|
|null|            null|            null|           null|  2|  Ph.D.|                EECS|UC Berkley|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+



##### Left Semi Joins

They do not actually include any values from the right DataFrame. If the value does exist, those rows will be kept in the result, even if there are duplicate keys in the left DataFrame. 

In [8]:
program.join(person, 
            person['graduate_program'] == program['id'], 
            'left_semi').show()

+---+-------+--------------------+----------+
| id| degree|          department|    school|
+---+-------+--------------------+----------+
|  0|Masters|School of Informa...|UC Berkley|
|  1|Masters|                EECS|UC Berkley|
+---+-------+--------------------+----------+



In [9]:
person.join(program, 
           person['graduate_program'] == program['id'],
           'left_semi').show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matel Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



##### Left Anti Joins

Left anti joins are the opposite of left semi joins.

In [10]:
program.join(person, 
            person['graduate_program'] == program['id'], 
            'left_anti').show()

+---+------+----------+----------+
| id|degree|department|    school|
+---+------+----------+----------+
|  2| Ph.D.|      EECS|UC Berkley|
+---+------+----------+----------+



##### Joins on Complex Types

Any expression is a valid join expression, assuming it returns a Boolean:

In [11]:
from pyspark.sql.functions import expr

person.selectExpr('id as personId', 'name', 'spark_status')\
.join(status, expr('array_contains(spark_status, id)')).show()

+--------+----------------+----------------+----+--------------+
|personId|            name|    spark_status|  id|        status|
+--------+----------------+----------------+----+--------------+
|       0|   Bill Chambers|           [100]| 100|   Contributor|
|       1|   Matel Zaharia| [500, 250, 100]| 500|Vice President|
|       1|   Matel Zaharia| [500, 250, 100]| 250|    PMC Member|
|       1|   Matel Zaharia| [500, 250, 100]| 100|   Contributor|
|       2|Michael Armbrust|      [250, 100]| 250|    PMC Member|
|       2|Michael Armbrust|      [250, 100]| 100|   Contributor|
|       3|            Luis|[250, 100, 1000]| 250|    PMC Member|
|       3|            Luis|[250, 100, 1000]| 100|   Contributor|
|       3|            Luis|[250, 100, 1000]|1000|           CEO|
+--------+----------------+----------------+----+--------------+



In [18]:
DATA_PATH = './data/'
movies_df = spark.read.csv(DATA_PATH + 'movies.csv', 
                           inferSchema=True, 
                           header=True, 
                           sep=',')
ratings_df = spark.read.csv(DATA_PATH + 'ratings.csv',
                           inferSchema=True,
                           header=True,
                           sep=',')

In [19]:
import pyspark.sql.functions as F
# Clean movies
clean_movies = movies_df.select('title', 'movieId',  
                 F.split(F.col('genres'), '\|')\
                .alias('genres'))\
.filter(F.array_contains(F.col('genres'), 'Horror') | 
        F.array_contains(F.col('genres'), 'Thriller'))
clean_movies.show(10, False)
# Join with ratings
rating_stats = ratings_df.groupBy(F.col('movieId'))\
.agg(F.count('*').alias('count'), 
     F.avg('rating').alias('mean_rating'))\
.filter(F.col('count') > 100)\
.join(clean_movies, on='movieId', how='left_semi')
rating_stats.show()

+-----------------------------------------+-------+-----------------------------------------+
|title                                    |movieId|genres                                   |
+-----------------------------------------+-------+-----------------------------------------+
|Heat (1995)                              |6      |[Action, Crime, Thriller]                |
|GoldenEye (1995)                         |10     |[Action, Adventure, Thriller]            |
|Dracula: Dead and Loving It (1995)       |12     |[Comedy, Horror]                         |
|Money Train (1995)                       |20     |[Action, Comedy, Crime, Drama, Thriller] |
|Get Shorty (1995)                        |21     |[Comedy, Crime, Thriller]                |
|Copycat (1995)                           |22     |[Crime, Drama, Horror, Mystery, Thriller]|
|Assassins (1995)                         |23     |[Action, Crime, Thriller]                |
|Twelve Monkeys (a.k.a. 12 Monkeys) (1995)|32     |[Mystery,

+-------+-----+------------------+
|movieId|count|       mean_rating|
+-------+-----+------------------+
|   1645|14346| 3.516589990241182|
|   1591| 6317|2.6416020262782967|
|   2122| 2825| 2.634513274336283|
|   3918| 1447|2.9595715272978578|
|   2366| 8162|3.4740872335211956|
|   1342| 3881|2.9637979902087093|
|   5300|  619| 3.608239095315024|
|   7982|  870| 3.607471264367816|
|   7993|  104|3.2788461538461537|
|  57370|  181|  2.81767955801105|
|    496|  423|3.2919621749408985|
|  31367|  207| 2.893719806763285|
| 166558|  205|3.0682926829268293|
|    463|  428|2.8119158878504673|
|   1127|19010| 3.666859547606523|
|  48780|20276| 4.078294535411324|
|  69481| 7319|3.7412214783440363|
|   1483| 3229| 3.121554660885723|
|    540| 5305|2.7241281809613573|
|  74452|  613|2.6557911908646004|
+-------+-----+------------------+
only showing top 20 rows

